In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not c

In [ ]:
import pandas as pd
# List of CSV file paths
csv_files = ['/content/syn_data_chapter_4.csv', '/content/chapter_1.csv','/content/chapter_2.csv','/content/chapter_17.csv']

# Verify and load CSV files
dataframes = [pd.read_csv(file) for file in csv_files]

# Combine dataframes
combined_data = pd.concat(dataframes, ignore_index=True)

# Save combined data to a new CSV file
combined_data.to_csv('combined_synthetic_data.csv', index=False)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
# Convert combined data to text format
texts = [" ".join([f"{col}: {val}" for col, val in row.items()]) for index, row in combined_data.iterrows()]

# Create Dataset object
dataset = Dataset.from_dict({"text": texts})
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
test_dataset = dataset['test']
# Tokenization function
def tokenize_function(examples):
    # Add labels to the inputs for language modeling
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/12610 [00:00<?, ? examples/s]

Map:   0%|          | 0/3153 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# Training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=100,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# PEFT configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

# Define the maximum sequence length
max_seq_length = 512

# SFTTrainer setup
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
# Fine-tune model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,2.266900
20,1.226800
30,0.769500
40,0.615300
50,0.462000
60,0.393000
70,0.346800
80,0.307900
90,0.279800
100,0.318600


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.6986665606498719, metrics={'train_runtime': 282.538, 'train_samples_per_second': 5.663, 'train_steps_per_second': 0.354, 'total_flos': 3.26413420855296e+16, 'train_loss': 0.6986665606498719, 'epoch': 0.12686330478908975})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/tokenizer.model',
 './results/added_tokens.json',
 './results/tokenizer.json')

In [ ]:
! pip install tabulate

In [ ]:
import pandas as pd

# Load abbreviations from a CSV file
def load_abbreviations(abbreviation_csv):
    data = pd.read_csv(abbreviation_csv)
    abbreviations = data["Abbreviation"].tolist()
    meanings = data["Meaning"].tolist()
    abbreviation_dict = dict(zip(abbreviations, meanings))
    return abbreviation_dict

abbreviation_csv = '/content/Medical Abbreviations.csv'  # Path to your abbreviations CSV file
abbreviation_dict = load_abbreviations(abbreviation_csv)


In [ ]:
def expand_abbreviations(prompt, abbreviation_dict):
    words = prompt.split()
    expanded_prompt = " ".join([abbreviation_dict.get(word, word) for word in words])
    return expanded_prompt

In [ ]:
import re
from tabulate import tabulate
#Normalize column names in the DataFrame
def normalize_columns(df):
    df.columns = [col.strip().lower() for col in df.columns]
    return df
def parse_conditions_and_examples(prompt, abbreviation_dict):
    filters = {}
    num_examples = None

    # Patterns to extract conditions and number of examples
    between_pattern = re.compile(r"(\w+)\s+between\s+(\d+\.?\d*)\s+and\s+(\d+\.?\d*)", re.IGNORECASE)
    specific_pattern = re.compile(r"(\w+):\s*([\w\s]+)", re.IGNORECASE)
    limit_pattern = re.compile(r"(\d+)\s+(patients|examples?)", re.IGNORECASE)

    # Find conditions
    between_matches = between_pattern.findall(prompt)
    specific_matches = specific_pattern.findall(prompt)
    limit_match = limit_pattern.search(prompt)

    for match in between_matches:
        key, lower, upper = match
        filters[key.strip().lower()] = (float(lower), float(upper))

    for match in specific_matches:
        key, value = match
        filters[key.strip().lower()] = value.strip().lower()

    # Check for disease mentions and map to primary_icd9_label
    disease_found = False
    disease_list = list(abbreviation_dict.values())
    for word in prompt.split():
        expanded_word = abbreviation_dict.get(word.lower(), word)
        if expanded_word.lower() in disease_list:
            filters['primary_icd9_label'] = expanded_word.lower()
            disease_found = True
            print(f"Filter added: primary_icd9_label = {expanded_word.lower()}")
            break

    if limit_match:
        num_examples = int(limit_match.group(1))
        print(f"Number of examples: {num_examples}")

    # If no disease was found in the prompt, avoid adding a primary_icd9_label filter
    if not disease_found:
        filters.pop('primary_icd9_label', None)

    return filters, num_examples

def apply_filters_to_data(filters, combined_data):
    filtered_data = combined_data.copy()
    print("Applying filters to data...")
    for key, value in filters.items():
        print(f"Filtering for {key}: {value}")
        if key in filtered_data.columns:
            if isinstance(value, tuple):
                lower, upper = value
                print(f"Filtering {key} between {lower} and {upper}")
                filtered_data = filtered_data[
                    (filtered_data[key].astype(float) >= lower) &
                    (filtered_data[key].astype(float) <= upper)
                ]
            else:
                print(f"Filtering {key} for value {value}")
                # Normalize column values and filter value
                filtered_data[key] = filtered_data[key].str.strip().str.lower()
                filtered_data = filtered_data[
                    filtered_data[key] == value.strip().lower()
                ]
    print(f"Filtered data:\n{filtered_data}")
    return filtered_data


def limit_number_of_examples(filtered_data, num_examples):
    if num_examples is not None:
        filtered_data = filtered_data.head(num_examples)
    return filtered_data


def format_response(filtered_data, columns):
    # Ensure it has the same columns as the original data
    response_df = filtered_data[columns]
    # Convert DataFrame to table format using tabulate
    response_table = tabulate(response_df, headers='keys', tablefmt='pretty')
    return response_table

def conversational_loop(columns, abbreviation_dict):
    while True:
        user_prompt = input("User: ")

        # Check for exit condition
        if user_prompt.lower() in ['exit', 'quit', 'q']:
            print("Exiting the conversation.")
            break

        # Expand abbreviations in the user prompt
        expanded_prompt = expand_abbreviations(user_prompt, abbreviation_dict)
        print(f"Expanded Prompt: {expanded_prompt}")

        # Generate response using the fine-tuned model
        inputs = tokenizer(expanded_prompt, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=512)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # # Print the raw response for debugging purposes
        # print(f"Raw LLM response: {response}")

        # Parse conditions and number of examples
        filters, num_examples = parse_conditions_and_examples(expanded_prompt, abbreviation_dict)
        # print(f"Filters: {filters}, Num examples: {num_examples}")

        # Apply filters to the data
        filtered_data = apply_filters_to_data(filters, combined_data)
        # print(f"Filtered Data after applying filters:\n{filtered_data}")

        # Limit the number of examples
        filtered_data = limit_number_of_examples(filtered_data, num_examples)

        # Format and print the response
        formatted_response = format_response(filtered_data, columns)
        print(f"LLM:\n{formatted_response}")






In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

In [ ]:
# Normalize column names
combined_data = normalize_columns(combined_data)

# Start conversational loop with the original columns and abbreviation dictionary
columns = combined_data.columns.tolist()
conversational_loop(columns, abbreviation_dict)

User: Give me a table with 4 patients
Expanded Prompt: Give me a table with 4 patients
Number of examples: 4
Filters: {}, Num examples: 4
Applying filters to data...
Filtered data:
       unnamed: 0 gender  age               ethnicity admission_type  \
0             0.0      M   52                   WHITE      EMERGENCY   
1             1.0      M   43                   WHITE      EMERGENCY   
2             2.0      F   72                   WHITE      EMERGENCY   
3             3.0      M   89                   WHITE      EMERGENCY   
4             4.0      M   66                   WHITE      EMERGENCY   
...           ...    ...  ...                     ...            ...   
15758         NaN      M   30        UNABLE TO OBTAIN      EMERGENCY   
15759         NaN      M   51                   WHITE      EMERGENCY   
15760         NaN      M   24        UNABLE TO OBTAIN      EMERGENCY   
15761         NaN      F   47  BLACK/AFRICAN AMERICAN       ELECTIVE   
15762         NaN      F   